In [1]:
import pronouncing
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, split, explode, udf
from pyspark.sql.types import ArrayType, StringType, IntegerType
from pyspark.sql import Row
from pyspark.sql import functions as F
import pandas as pd
import re

In [2]:
rowData = spark.read.csv("/FileStore/tables/hadoop_final_project/lyrics.csv", inferSchema=True, header = True, multiLine=True)

In [3]:
rowData.show(n=5)

In [4]:
data_array_lyrics = rowData.withColumn(
    "lyrics",
    split(col("lyrics"), "\n").cast(ArrayType(StringType())).alias("lyrics")
)
data_array_lyrics.show(n=5)

In [5]:
data_line = data_array_lyrics.withColumn("lyrics", explode(data_array_lyrics.lyrics))
data_line.show(n=5)

In [6]:
def phone_phrase(phrase):
  s = filter(bool, re.split(r'\W+', phrase.lower()))
  phone = []
  for p in s:
    tmp = pronouncing.phones_for_word(p)
    if len(tmp) != 0:
      phone.append(tmp[0])
  return phone
pp_udf = udf(phone_phrase, ArrayType(StringType()))

In [7]:
data_phone = data_line.withColumn("phoneme", pp_udf(data_line['lyrics']))
data_phone.show(n=5)

In [8]:
def length_phrase(phrase):
  s = filter(bool, re.split(r'\W+', phrase.lower()))
  return len(s)
len_udf = udf(length_phrase, IntegerType())
data_phone_len = data_phone.withColumn('length', len_udf(data_phone['lyrics']))
data_phone_len.show(n=5)